In [86]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

In [87]:
datasets = {
    "reduce_mrmr_instances": pd.read_csv('./data/df_reduce_mrmr_instances.csv'),
    "reduce_mrmr_instances_hard": pd.read_csv('./data/df_reduce_mrmr_instances_hard.csv'),
    "reduce_RFC_instances": pd.read_csv('./data/df_reduce_RFC_instances.csv'),
    "reduce_RFC_instances_hard": pd.read_csv('./data/df_reduce_RFC_instances_hard.csv'),
    "reduce_mrmr_instances_GLVQ": pd.read_csv('./data/df_reduce_mrmr_instances_GLVQ.csv'),
    "reduce_RFC_instances_GLVQ": pd.read_csv('./data/df_reduce_RFC_instances_GLVQ.csv'),
    # "train_data": pd.read_csv('./data/train_data.csv')
    "test_data" : pd.read_csv('./data/test_data.csv')
}

In [88]:
# Seleccionamos los datos de prueba del conjunto de datos
data_prueba = datasets['test_data']

# Separar las características (X) de la variable objetivo (y)
x = data_prueba.drop(columns=['Class'])
y_test_total = data_prueba['Class']

# Filtramos las columnas relevantes según mRMR
columns_to_keep = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']
x_reduced = x[columns_to_keep]

# Crear el scaler para normalizar los datos
scaler = MinMaxScaler()

# Normalizamos las columnas seleccionadas
x_reduced['Amount'] = scaler.fit_transform(x_reduced[['Amount']])
x_reduced['Time'] = scaler.fit_transform(x_reduced[['Time']])

C:\Users\joses\AppData\Local\Temp\ipykernel_1360\1107825650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_reduced['Amount'] = scaler.fit_transform(x_reduced[['Amount']])
C:\Users\joses\AppData\Local\Temp\ipykernel_1360\1107825650.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_reduced['Time'] = scaler.fit_transform(x_reduced[['Time']])


## Logistic Regresion con mrMr clusterCentroid_soft

In [89]:
df_reduce_mrmr_instances = datasets["reduce_mrmr_instances"]
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = [
    # Configuración para 'elasticnet'
    {
        'penalty': ['elasticnet'],
        'solver': ['saga'],  # Solo 'saga' es compatible con elasticnet
        'C': [0.1, 1, 10],
        'l1_ratio': [0.5],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    },
    # Configuración para 'l1' y 'l2'
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],  # Solvers compatibles con l1 y l2
        'C': [0.1, 1, 10],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    }
]


# Configurar el modelo de regresión logística
log_reg = LogisticRegression()

# Usar GridSearchCV para buscar la mejor combinación de parámetros
# Dada la naturaleza desbalanceada de los datos
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1, verbose=1)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor puntaje
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Mejores parámetros: {'C': 0.1, 'class_weight': 'balanced', 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.0001}
Mejor puntuación de validación cruzada: 0.8377987567249356


## Ejecutamos para el conjunto de prueba

In [90]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_reduced)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[27661   771]
 [   14    35]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.99     28432
Fraudulentas       0.04      0.71      0.08        49

    accuracy                           0.97     28481
   macro avg       0.52      0.84      0.53     28481
weighted avg       1.00      0.97      0.98     28481



## Logistic Regresion con mrMr clusterCentroid_hard

In [91]:
df_reduce_mrmr_instances_hard = datasets["reduce_mrmr_instances_hard"]
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = [
    # Configuración para 'elasticnet'
    {
        'penalty': ['elasticnet'],
        'solver': ['saga'],  # Solo 'saga' es compatible con elasticnet
        'C': [0.1, 1, 10],
        'l1_ratio': [0.5],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    },
    # Configuración para 'l1' y 'l2'
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],  # Solvers compatibles con l1 y l2
        'C': [0.1, 1, 10],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    }
]


# Configurar el modelo de regresión logística
log_reg = LogisticRegression()

# Usar GridSearchCV para buscar la mejor combinación de parámetros
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor puntaje
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Mejores parámetros: {'C': 0.1, 'class_weight': None, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.001}
Mejor puntuación de validación cruzada: 0.8559584457097194


## Ejecutamos para el conjunto de prueba

In [92]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(x_reduced)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[27575   857]
 [   14    35]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.98     28432
Fraudulentas       0.04      0.71      0.07        49

    accuracy                           0.97     28481
   macro avg       0.52      0.84      0.53     28481
weighted avg       1.00      0.97      0.98     28481



## Logistic Regresion con mrMr y GLVQ

In [93]:
# No se van a usar los acercamientos con GLVQ dada la imposibilidad con el entrenamiento del modelo de regresion

## Logistic Regresion con RFC clusterCentroid_soft

In [94]:
df_reduce_RFC_instances = datasets["reduce_RFC_instances"]
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = [
    # Configuración para 'elasticnet'
    {
        'penalty': ['elasticnet'],
        'solver': ['saga'],  # Solo 'saga' es compatible con elasticnet
        'C': [0.1, 1, 10],
        'l1_ratio': [0.5],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    },
    # Configuración para 'l1' y 'l2'
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],  # Solvers compatibles con l1 y l2
        'C': [0.1, 1, 10],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    }
]


# Configurar el modelo de regresión logística
log_reg = LogisticRegression()

# Usar GridSearchCV para buscar la mejor combinación de parámetros
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor puntaje
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Mejores parámetros: {'C': 1, 'class_weight': None, 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 200, 'penalty': 'elasticnet', 'solver': 'saga', 'tol': 0.0001}
Mejor puntuación de validación cruzada: 0.925162321446409


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Ejecutamos para el conjunto prueba

In [95]:
from sklearn.preprocessing import MinMaxScaler
# Cogemos los datos de test y les eliminamos las cracterísticas que no necesitamos
X = data_prueba.drop(columns=['Class'])
y_test_final = data_prueba['Class']
### Version con RandomForestClassifier
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']
X_reduce_RFC = X[columns_to_keep_RFC]

print(y_test_final.value_counts())

Class
0    28432
1       49
Name: count, dtype: int64


In [96]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_reduce_RFC)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[28133   299]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      0.99     28432
Fraudulentas       0.13      0.90      0.22        49

    accuracy                           0.99     28481
   macro avg       0.56      0.94      0.61     28481
weighted avg       1.00      0.99      0.99     28481



## Logistic Regresion con RFC clusterCentroid_hard

In [97]:
df_reduce_RFC_instances_hard = datasets["reduce_RFC_instances_hard"]
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = [
    # Configuración para 'elasticnet'
    {
        'penalty': ['elasticnet'],
        'solver': ['saga'],  # Solo 'saga' es compatible con elasticnet
        'C': [0.1, 1, 10],
        'l1_ratio': [0.5],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    },
    # Configuración para 'l1' y 'l2'
    {
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],  # Solvers compatibles con l1 y l2
        'C': [0.1, 1, 10],
        'tol': [0.0001, 0.001],
        'max_iter': [100, 200],
        'class_weight': [None, 'balanced'],
        'fit_intercept': [True],
    }
]


# Configurar el modelo de regresión logística
log_reg = LogisticRegression()

# Usar GridSearchCV para buscar la mejor combinación de parámetros
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor puntaje
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros:", best_params)
print("Mejor puntuación de validación cruzada:", best_score)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Mejores parámetros: {'C': 10, 'class_weight': 'balanced', 'fit_intercept': True, 'l1_ratio': 0.5, 'max_iter': 200, 'penalty': 'elasticnet', 'solver': 'saga', 'tol': 0.0001}
Mejor puntuación de validación cruzada: 0.925162321446409


c:\Users\joses\miniconda3\envs\pre_procesamiento\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


## Ejecutamos para el conjunto prueba

In [98]:
# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_reduce_RFC)
test_accuracy = accuracy_score(y_test_total, y_pred)

# Calcular la matriz de confusión y el reporte de clasificación
conf_matrix = confusion_matrix(y_test_total, y_pred)
report = classification_report(y_test_total, y_pred, target_names=['Correctas', 'Fraudulentas'])

# Mostrar la matriz de confusión y el reporte
print("\nMatriz de Confusión:")
print(conf_matrix)

print("\nReporte de Clasificación:")
print(report)


Matriz de Confusión:
[[28120   312]
 [    5    44]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      0.99     28432
Fraudulentas       0.12      0.90      0.22        49

    accuracy                           0.99     28481
   macro avg       0.56      0.94      0.61     28481
weighted avg       1.00      0.99      0.99     28481



## Logistic Regresion con RFC y GLVQ

In [99]:
# No se va a usar el acercamiento con GLVQ dada la imposibilidad con el entrenamiento del modelo de regresion

# Conclusiones

# Regresión logística - Elección mejor preprocesamiento

|                        | Precisión train | Precisión test | f1 test |
|------------------------|-----------------|----------------|---------|
| **mrMr - CC soft**     | **0.84**       | **0.52**       | **0.53**|
| **mrMr - CC hard**     | **0.85**       | **0.52**       | **0.53**|
| **RFC - CC soft**      | **0.93**       | **0.56**       | **0.61**|
| **RFC - CC hard**      | **0.93**       | **0.56**       | **0.61**|




La combinación de modelos y estrategias de preprocesamiento muestra variaciones claras en las métricas de precisión y el comportamiento de los modelos frente a datos desbalanceados.
Los métodos "hard" tienden a favorecer la mejora en las predicciones para la clase minoritaria (fraudulentas), a costa de una ligera pérdida en el rendimiento de la clase mayoritaria.

## Matriz de Confusión y Clasificación:

### mrMr - CC soft:

Muestra una precisión muy alta para la clase mayoritaria (correctas), pero un rendimiento pobre para la clase minoritaria (fraudulentas).
Aunque tiene una tasa de recall del 71% en la clase minoritaria, la precisión del 4% implica un alto número de falsos positivos.
Este modelo destaca más en mantener la precisión para las predicciones generales que en equilibrar clases.

### mrMr - CC hard:

Similar a la versión "soft", pero con mayor recall en fraudulentas debido al ajuste del preprocesamiento. Esto refuerza las detecciones para la clase minoritaria, aunque no mejora significativamente la precisión.

### RFC - CC soft:

Muy similar al modelo anterior (mrMr soft) en términos de matriz de confusión y métricas. La precisión sigue siendo baja en la clase minoritaria.
Aunque los resultados son consistentes en la clase mayoritaria, las predicciones para fraudulentas tienen pocas mejoras.

### RFC - CC hard:

Este modelo logra el mejor equilibrio entre clases, con un recall del 90% y una precisión mejorada al 12% para la clase minoritaria. Aunque sigue siendo baja, representa un avance considerable comparado con las otras configuraciones.
El f1-score de 0.61 en general refleja su capacidad para detectar correctamente ambas clases.

### Conclusiones:
El desbalance de clases (con una clase minoritaria que representa solo el 0.17% del total) es un desafío importante para los modelos. Los métodos de preprocesamiento "hard" parecen abordar mejor este problema.

Las matrices de confusión muestran que los modelos "soft" generan un número alto de falsos negativos para la clase minoritaria (fraudulentas), mientras que los modelos "hard" logran reducirlos significativamente.
